# Set up

The following package will download the TSV files from IMDB website and store the data into a SQLite database (requires 10GB of disk space)


In [ ]:
!pip install imdb-sqlite

In [ ]:
%%time
!imdb-sqlite

In [ ]:
# Need this package for hadling country ISO codes
!pip install pycountry

In [ ]:
# Imports
from matplotlib import cm  # Color maps
from sklearn.feature_extraction.text import CountVectorizer  # Convert a collection of text documents to a matrix of token counts
import re
import time
import sqlite3
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


In [ ]:
# Some tweaks

warnings.filterwarnings('ignore')  # Don't show warnings
sns.set_theme(style='whitegrid')

# Database structure analysis

In [ ]:
# Connect to databse
conn = sqlite3.connect('imdb.db')

In [ ]:
# Retrieve tables' names
tables = pd.read_sql_query('SELECT name AS "Table_name" FROM sqlite_master WHERE type="table"', conn)
tables.head()

In [ ]:
# Retrieve each table's schema

tables_names = tables['Table_name'].values.tolist()

for table_name in tables_names:
    schema = pd.read_sql_query(f'PRAGMA TABLE_INFO({table_name})', conn)
    print(f'Schema for table {table_name}:')
    display(schema)
    print(f'{50 * "_"}\n\n')

# 1. The most common content categories in IMDB

In [ ]:
# Retrieve the data
query = 'SELECT type, COUNT(*) AS abs_frequency FROM titles GROUP BY type ORDER BY abs_frequency DESC'
dataframe = pd.read_sql_query(query, conn)
display(dataframe)

In [ ]:
# Remove tvEpisode type
dataframe = dataframe.drop(labels=0, axis=0)

In [ ]:
# Add the relative_frequency
sum_abs_freq = dataframe['abs_frequency'].sum()
dataframe['rel_frequency'] = (dataframe['abs_frequency'] / sum_abs_freq) * 100
display(dataframe)

Now let's create a graph with 5 categories:

4 categories with the the biggest amount of titles.

1 category with the others


In [ ]:
# Others category
others = {}
others['type'] = 'others'
others['abs_frequency'] = dataframe.iloc[4:]['abs_frequency'].sum()
others['rel_frequency'] = dataframe.iloc[4:]['rel_frequency'].sum()
others

In [ ]:
# The common title categories dataframe
dataframe = dataframe.head(4)
dataframe = dataframe.append(others, ignore_index=True)  # Append others category
dataframe = dataframe.sort_values(by='rel_frequency', ascending=False)
dataframe

In [ ]:
# Labels
labels = []

for i in dataframe.index:
    cont_type = dataframe.loc[i]["type"]
    cont_type = re.sub(r"(\w)([A-Z])", r"\1 \2", cont_type) # Add space before capital letters
    cont_type = cont_type[0].upper() + cont_type[1:] # Uppercase the first letter without changing the rest

    rel_freq = round(dataframe.loc[i]["rel_frequency"], 2)

    labels.append(f'{cont_type} {rel_freq}%')

labels

In [ ]:
# Plot

# Set color map
# https://matplotlib.org/stable/tutorials/colors/colormaps.html
cs = cm.tab10(range(100))

plt.pie(dataframe['rel_frequency'], colors = cs, labeldistance = 1, radius = 3, wedgeprops = dict(width = 0.8))
plt.legend(labels = labels, loc = 'center', fontsize = 12)
plt.title(label = 'Content Categories', loc = 'center', fontdict = {'fontsize':20,'fontweight':20})
plt.show()

# 2. Percentage of movies per genre

In [ ]:
# Retrieve movie genres and sort them
query = 'SELECT genres FROM titles WHERE type="movie"'
dataframe = pd.read_sql_query(query, conn)
display(dataframe)

In [ ]:
# Lowercase genres
dataframe['genres'] = dataframe['genres'].str.lower()

In [ ]:
# Function for removing missing values

def rm_miss_values(df: pd.DataFrame, column: str, miss_indicator: str = '\\n') -> pd.DataFrame:
    if column != None:
        df[column].replace(miss_indicator, np.nan, inplace=True)
        cleared_df = df.dropna(subset=[column])
    else:
        df.replace(miss_indicator, np.nan, inplace=True)
        cleared_df = df.dropna()
    return cleared_df

In [ ]:
# Remove missing values
genres_df = rm_miss_values(dataframe, 'genres')
display(genres_df)

#### How CountVectorizer works:

![How CountVectorizer works](https://mlwhiz.com/images/countvectorizer.png)

In [ ]:
# Function that creates a dataframe with unique genres and the respectively absolute frequency

def df_uniq_genres_abs_freq(df: pd.DataFrame, column: str, abs_freq=True) -> pd.DataFrame:
    # Match words with lenght > 1 and with - or not
    pattern = '(?u)\\b\\w[\\w-]+\\b'

    # Deals with multi-categorical data
    vect = CountVectorizer(token_pattern=pattern).fit(df[column])

    # Create Document-term matrix.
    bag_genres = vect.transform(df[column])

    # Create array of unique genres
    unique_genres = vect.get_feature_names()

    if abs_freq:
        # Sum sparse matrix columns
        genres_abs_freq = bag_genres.toarray().sum(axis=0)

        # Return DataFrame with genres and absolute frequencies
        return pd.DataFrame(zip(unique_genres, genres_abs_freq), columns=['genre', 'abs_freq'])
    else:
        return pd.DataFrame(unique_genres, columns=['genre'])

In [ ]:
# Create final dataframe
genres_df = df_uniq_genres_abs_freq(genres_df, column='genres')
genres_df

In [ ]:
# Sum of absolute frequencies
freq_total_sum = genres_df['abs_freq'].sum()
freq_total_sum

In [ ]:
# Sort by absolute frequency
genres_df = genres_df.sort_values(by='abs_freq', ascending=False)
genres_df

In [ ]:
# Add relative frequencies to dataframe
genres_df['rel_freq'] = [(abs_freq / freq_total_sum) * 100 for abs_freq in genres_df['abs_freq']]
genres_df

In [ ]:
# Plot

plt.figure(figsize=(16, 8))
sns.barplot(x=genres_df['rel_freq'].tolist(), y=genres_df['genre'].tolist(),
            orient='h', palette='terrain')
plt.ylabel('Gender')
plt.xlabel('Percentual of movies (%)')
plt.title('Percentage of movie genres')
plt.show()


# 3. Movie ratings median per genre

In [ ]:
query = 'SELECT rating, genres AS genre FROM ratings JOIN titles ON ratings.title_id = titles.title_id WHERE type = "movie"'

In [ ]:
genre_df = pd.read_sql_query(query, conn)
display(genre_df)

In [ ]:
genre_df['genre'] = genre_df['genre'].str.lower()
display(genre_df)

In [ ]:
# Remove missing values
genre_df = rm_miss_values(genre_df, column='genre')
display(genre_df)

In [ ]:
genre_df = df_uniq_genres_abs_freq(genre_df, column='genre')
genre_df

In [ ]:
# Compute the rating median for each gender
def genre_rating_median(genres: list, conn: sqlite3.Connection) -> dict:
    genre_ratings = {}

    for genre in genres:
        query = f'SELECT rating FROM ratings JOIN titles ON ratings.title_id = titles.title_id WHERE genres LIKE "%{genre}%" AND type = "movie"'
        result = pd.read_sql_query(query, conn)
        genre_ratings[genre] = np.median(result['rating'])

    return genre_ratings

In [ ]:
genre_ratings = genre_rating_median(genre_df['genre'], conn)
genre_ratings

In [ ]:
# Game Show, Talk Show, Reality TV and News aren't movie genres, so lets remove them

for genre in ['game-show', 'talk-show', 'news', 'reality-tv']:
    genre_ratings.pop(genre)
    genre_df = genre_df[genre_df.genre != genre]

display(genre_df)
genre_ratings

In [ ]:
# Create column rating and sort them

genre_df['rating'] = genre_ratings.values()
genre_df = genre_df.sort_values(by='rating', ascending=False)
genre_df

In [ ]:
# Plot

plt.figure(figsize=(18, 10))
sns.barplot(x=genre_df['rating'], y=genre_df['genre'], orient='h')

# Plot Text
for i in range(len(genre_df.index)):
    plt.text(x=4, y=i + .22,
             s=f'{genre_df["abs_freq"][genre_df.index[i]]} titles')

    rating = genre_df['rating'][genre_df.index[i]]
    plt.text(x=rating + .05, y=i + .22, s=rating)

plt.xlabel('Rating Median')
plt.ylabel('Genre')
plt.title('\nGenre Rating Median\n')
plt.show()

# 4. Median of movie ratings by premiered year

In [ ]:
query = 'SELECT rating, premiered FROM ratings JOIN titles ON ratings.title_id = titles.title_id WHERE type = "movie" ORDER BY premiered'

In [ ]:
df = pd.read_sql_query(query, conn)
display(df)

In [ ]:
# Clear empty values

df.dropna(axis=0, inplace=True)
df

In [ ]:
# Computes the median

ratings = {}
for i in set(df['premiered']):
    ratings_df = df[df['premiered'] == i]['rating']
    ratings[i] = np.median(ratings_df)

ratings

In [ ]:
# Plot

plt.figure(figsize=(18, 8))
sns.lineplot(x=ratings.keys(), y=ratings.values())
plt.title('Median of movie ratings by year')
plt.xlabel('Premiered year')
plt.ylabel('Ratings median')
plt.show()

# 5. Number of titles premiered each year by genre

In [ ]:
query = 'SELECT genres FROM titles WHERE type = "movie"'

In [ ]:
df = pd.read_sql_query(query, conn)
df

In [ ]:
# Clear data

# Lowercase genres
df['genres'] = df['genres'].str.lower()

df = rm_miss_values(df, column=None)
df

In [ ]:
# Compute unique genres from df

genres_df = df_uniq_genres_abs_freq(df, 'genres', abs_freq=False)
genres_df

In [ ]:
# Game Show, Talk Show, Reality TV and News aren't movie genres, so lets remove them

for genre in ['game-show', 'talk-show', 'news', 'reality-tv']:
    genres_df = genres_df[genres_df.genre != genre]

display(genres_df)

In [ ]:
# Top 5 most common genres

genre_top5 = {}

for genre in genres_df['genre']:
    query = f'SELECT COUNT(*) AS abs_freq FROM titles WHERE genres LIKE "%{genre}%" AND type = "movie" AND premiered <= 2022'
    genre_top5[genre] = pd.read_sql_query(query, conn).iloc[0]['abs_freq']

# Sort and return top 5
genre_top5 = dict(sorted(genre_top5.items(), key=lambda item: item[1], reverse=True)[:5])
genre_top5

In [ ]:
# Plot
plt.figure(figsize=(18,8))

# Retrive premiered movies by genre and year
for genre in genre_top5:
    query = f'SELECT premiered, COUNT(*) AS abs_frequency FROM titles WHERE genres LIKE "%{genre}%" AND premiered NOT NULL AND premiered <= 2022 AND type = "movie" GROUP BY premiered'
    genres_df = pd.read_sql_query(query, conn)
    sns.lineplot(x=genres_df['premiered'], y=genres_df['abs_frequency'])


plt.title("\nNumber of titles premiered each year by genre\n")
plt.ylabel("Number of movies")
plt.xlabel("Premiered year")
plt.legend(labels=genre_top5)
plt.show()